# Example of rigid-rotor calculations for water molecule

In [1]:
from richmol.rot.asymtop import RotStates
from richmol.rot.cartens import CartTensor

In [2]:
xyz = (
    "bohr", "c2v",
    "O", 0.00000000, 0.00000000, 0.12395915,
    "H", 0.00000000, -1.43102686, -0.98366080,
    "H", 0.00000000,  1.43102686, -0.98366080,
)

jmax = 10
states = RotStates.from_geometry(jmax, xyz)

# molecular-frame dipole moment in au
dip_mol = [0, 0, -0.7288]

# molecular-frame polarizability in au
pol_mol = [[9.1369, 0, 0], [0, 9.8701, 0], [0, 0, 9.4486]]

dip_lab = CartTensor(states, dip_mol)
pol_lab = CartTensor(states, pol_mol)


Compute rigid-rotor solutions using molecular geometry as input
Cartesian units: bohr, conversion to Angstrom: 0.529177210544
  Atom           Mass (u)              X (Å)              Y (Å)              Z (Å)
     O    15.994914619260     0.000000000000     0.000000000000     0.065596357218
     H     1.007825031898     0.000000000000    -0.757266801988    -0.520530878265
     H     1.007825031898     0.000000000000     0.757266801988    -0.520530878265
Symmetry group: c2v
solve for J = 0
solve for J = 1
solve for J = 2
solve for J = 3
solve for J = 4
solve for J = 5
solve for J = 6
solve for J = 7
solve for J = 8
solve for J = 9
solve for J = 10
  J Irrep    i         Energy (E)      (J,k,tau,Irrep)     c_max²
  0    A1    0         0.00000000      (0, 0, 0, 'A1')    1.00000
  1    A2    0        41.99637175      (1, 1, 1, 'A2')    1.00000
  1    B1    0        36.93165367      (1, 0, 1, 'B1')    1.00000
  1    B2    0        24.10374319      (1, 1, 0, 'B2')    1.00000
  2    A1    0